In [2]:
# Import our dependencies
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import sklearn as skl
import tensorflow as tf
import numpy as np

2024-06-03 14:39:18.805336: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
import os
import nbformat
from nbconvert import PythonExporter
from IPython.core.interactiveshell import InteractiveShell
import sys
from io import StringIO

def load_notebook(path):
    # Load the notebook
    with open(path, 'r', encoding='utf-8') as f:
        nb = nbformat.read(f, as_version=4)
    
    # Convert the notebook to a Python script
    exporter = PythonExporter()
    source, _ = exporter.from_notebook_node(nb)
    
    # Create an interactive shell to execute the script
    shell = InteractiveShell.instance()
    
    # Suppress standard output and error
    old_stdout = sys.stdout
    old_stderr = sys.stderr
    sys.stdout = StringIO()
    sys.stderr = StringIO()
    
    try:
        # Execute the Python script
        exec(source, shell.user_ns)
    finally:
        # Restore standard output and error
        sys.stdout = old_stdout
        sys.stderr = old_stderr
    
    return shell.user_ns

# Specify the path to your notebook
notebook_path = 'sql.ipynb'

# Load the notebook and extract the namespace
notebook_ns = load_notebook(notebook_path)

# Access each of the dataframes
model_eight_df = notebook_ns['model_eight_df']

# Print the head of each dataframe
model_eight_df.head()


,popularity,is_playable,danceability,energy,loudness,acousticness,valence,tempo,duration_ms,Monday,...,instrumentalness_below_5,is_live,is_not_live,artists_popularities,artists_followers,artists_followers,album,compilation,single,genre_numeric


In [4]:
# Preprocessing the data

# Define the features (X) and target (y)
X = model_eight_df.drop(columns=['popularity'])
y = model_eight_df['popularity']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)



ValueError: With n_samples=0, test_size=0.25 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

ValueError: could not convert string to float: '[46]'

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer
nn = tf.keras.models.Sequential()
number_input_features = len(X_train_scaled[0])

hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                2160      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 4621 (18.05 KB)
Trainable params: 4621 (18.05 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=20)



Epoch 1/20
4963/4963 [==============================] - 7s 1ms/step - loss: -28818212864.0000 - accuracy: 0.0000e+00
Epoch 2/20
4963/4963 [==============================] - 8s 2ms/step - loss: -37628653568.0000 - accuracy: 0.0000e+00
Epoch 3/20
4963/4963 [==============================] - 8s 2ms/step - loss: -48069636096.0000 - accuracy: 0.0000e+00
Epoch 4/20
4963/4963 [==============================] - 8s 2ms/step - loss: -60269961216.0000 - accuracy: 0.0000e+00
Epoch 5/20
4963/4963 [==============================] - 8s 2ms/step - loss: -74388283392.0000 - accuracy: 0.0000e+00
Epoch 6/20
4963/4963 [==============================] - 8s 2ms/step - loss: -90536419328.0000 - accuracy: 0.0000e+00
Epoch 7/20
4963/4963 [==============================] - 8s 2ms/step - loss: -108882632704.0000 - accuracy: 0.0000e+00
Epoch 8/20
4963/4963 [==============================] - 8s 2ms/step - loss: -129518510080.0000 - accuracy: 0.0000e+00
Epoch 9/20
4963/4963 [==============================] - 8s 2ms

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


1655/1655 - 2s - loss: -6.3785e+11 - accuracy: 0.0000e+00 - 2s/epoch - 1ms/step
Loss: -637850550272.0, Accuracy: 0.0


In [ ]:
# Export our model to HDF5 file
nn.save('Models/PopularityModel-8.h5')

/Users/CaseyWaggoner/miniconda3/envs/dev/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
